In [1]:
import re
import pandas as pd
import networkx as nx
import plotly.graph_objects as go

# Regex para parsing do Apache
log_pattern = r'(?P<ip>\d+\.\d+\.\d+\.\d+) - - \[(?P<date>.*?)\] "(?P<method>\w+) (?P<url>\S+) HTTP/\d\.\d" (?P<status>\d{3}) (?P<size>\d+|-) "(?P<referrer>.*?)" "(?P<user_agent>.*?)"'

# Caminho do seu arquivo de log
log_file = 'logs-invasao.log'

# Carregar logs em DataFrame
entries = []
with open(log_file, 'r', encoding='utf-8', errors='ignore') as f:
    for line in f:
        m = re.match(log_pattern, line)
        if m:
            entries.append(m.groupdict())

df = pd.DataFrame(entries)

# Converter datetime
df['datetime'] = pd.to_datetime(df['date'], format='%d/%b/%Y:%H:%M:%S %z')

# Criar coluna com granularidade de 1 hora
df['minute'] = df['datetime'].dt.floor('60min')  # floor to minute

# Verificar resultado
print("Registros lidos:", len(df))
print("Intervalos encontrados:", df['minute'].nunique())

print("Quantidade de registros por hora:")
print(df['minute'].value_counts().sort_index())

# Lista de minutos ordenados
all_minutes = sorted(df['minute'].unique())

# Gerar grafos e capturar dados de cada frame
frames_data = []

for minute in all_minutes:
    df_minute = df[df['minute'] == minute]
    
    # Criar grafo bipartido IP ↔ URL
    G = nx.Graph()
    for _, row in df_minute.iterrows():
        ip = row['ip']
        url = row['url']
        G.add_node(ip, bipartite=0)
        G.add_node(url, bipartite=1)
        G.add_edge(ip, url)
    
    # Calcular layout (usar spring layout para visual mais estável)
    pos = nx.spring_layout(G, k=0.5, seed=42)
    
    # Separar nós
    ips = [n for n, d in G.nodes(data=True) if d['bipartite'] == 0]
    urls = [n for n in G if n not in ips]
    
    # Adicionar dados para plotly
    edge_x = []
    edge_y = []
    for e in G.edges():
        x0, y0 = pos[e[0]]
        x1, y1 = pos[e[1]]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]
    
    # Nós IPs
    ip_x = [pos[n][0] for n in ips]
    ip_y = [pos[n][1] for n in ips]
    
    # Nós URLs
    url_x = [pos[n][0] for n in urls]
    url_y = [pos[n][1] for n in urls]
    
    # Guardar frame
    frames_data.append({
        'minute': minute,
        'edge_x': edge_x,
        'edge_y': edge_y,
        'ip_x': ip_x,
        'ip_y': ip_y,
        'ip_labels': ips,
        'url_x': url_x,
        'url_y': url_y,
        'url_labels': urls
    })

# Criar primeiro frame (para inicializar)
initial = frames_data[0]

# Criar figura
fig = go.Figure(
    data=[
        go.Scatter(
            x=initial['edge_x'],
            y=initial['edge_y'],
            mode='lines',
            line=dict(color='gray', width=1),
            hoverinfo='none'
        ),
        go.Scatter(
            x=initial['ip_x'],
            y=initial['ip_y'],
            mode='markers+text',
            marker=dict(size=10, color='lightblue'),
            text=initial['ip_labels'],
            textposition='top center',
            name='IPs'
        ),
        go.Scatter(
            x=initial['url_x'],
            y=initial['url_y'],
            mode='markers+text',
            marker=dict(size=10, color='salmon'),
            text=initial['url_labels'],
            textposition='top center',
            name='URLs'
        )
    ],
    layout=go.Layout(
        title_text=f"Grafo Temporal Frames por Hora (LOG INVASÃO) - {frames_data[0]['minute'].strftime('%d-%m-%Y %H:%M')}",
        width=1200,       # LARGURA DA FIGURA
        height=800,      # ALTURA DA FIGURA
        #title=f'Grafo Temporal - {initial["minute"]}',
        showlegend=True,
        updatemenus=[
            dict(
                type="buttons",
                buttons=[dict(label="Play",
                              method="animate",
                              args=[None, {"frame": {"duration": 1000, "redraw": True},
                                           "fromcurrent": True}]),
                         dict(label="Pause",
                              method="animate",
                              args=[[None], {"frame": {"duration": 0, "redraw": False},
                                             "mode": "immediate"}])]
            )
        ]
    ),
    frames=[
        go.Frame(
            data=[
                go.Scatter(
                    x=frame['edge_x'],
                    y=frame['edge_y'],
                    mode='lines',
                    line=dict(color='gray', width=1),
                    hoverinfo='none'
                ),
                go.Scatter(
                    x=frame['ip_x'],
                    y=frame['ip_y'],
                    mode='markers+text',
                    marker=dict(size=10, color='lightblue'),
                    text=frame['ip_labels'],
                    textposition='top center',
                    name='IPs'
                ),
                go.Scatter(
                    x=frame['url_x'],
                    y=frame['url_y'],
                    mode='markers',
                    marker=dict(size=10, color='salmon'),
                    text=frame['url_labels'],
                    textposition='top center',
                    name='URLs'
                )
            ],
            name=str(frame['minute']),
            layout=go.Layout(
                title_text=f"Grafo Temporal Frames por Hora (LOG INVASÃO) - {frame['minute'].strftime('%d-%m-%Y %H:%M')}",
                width=1200,     # LARGURA DA FIGURA
                height=800      # ALTURA DA FIGURA
            )
        )
        for frame in frames_data
    ]
)

# Mostrar figura interativa
fig.show()

fig.write_html("Grafo-Temporal-Interativo-IPxURL.html")

Registros lidos: 4851
Intervalos encontrados: 49
Quantidade de registros por hora:
minute
2025-05-10 07:00:00-03:00      14
2025-05-10 08:00:00-03:00      23
2025-05-10 09:00:00-03:00      32
2025-05-10 10:00:00-03:00     427
2025-05-10 11:00:00-03:00    1274
2025-05-10 12:00:00-03:00      43
2025-05-10 13:00:00-03:00      25
2025-05-10 14:00:00-03:00      25
2025-05-10 15:00:00-03:00      37
2025-05-10 16:00:00-03:00      27
2025-05-10 17:00:00-03:00      14
2025-05-10 18:00:00-03:00      18
2025-05-10 19:00:00-03:00      28
2025-05-10 20:00:00-03:00      26
2025-05-10 21:00:00-03:00     201
2025-05-10 22:00:00-03:00      35
2025-05-10 23:00:00-03:00      15
2025-05-11 00:00:00-03:00      10
2025-05-11 01:00:00-03:00      17
2025-05-11 02:00:00-03:00      24
2025-05-11 03:00:00-03:00     497
2025-05-11 04:00:00-03:00      28
2025-05-11 05:00:00-03:00      20
2025-05-11 06:00:00-03:00      22
2025-05-11 07:00:00-03:00      39
2025-05-11 08:00:00-03:00      52
2025-05-11 09:00:00-03:00 